In [7]:
from keras.models import Sequential, Model
from keras.applications import VGG16
from keras.layers import (Dense, BatchNormalization, Convolution2D, 
                         MaxPooling2D, Dropout, Input, Flatten)
from keras.optimizers import Adam
from keras_utilities import Vgg16, get_batches, onehot, load_array, save_array

In [2]:
mod = Vgg16()

In [3]:
max_conv_layer = [i for i,l in enumerate(mod.model.layers) if isinstance(l, Convolution2D)][-1]

In [4]:
trn_batches = get_batches("data/train", shuffle=False, batch_size=64)
val_batches = get_batches("data/valid", shuffle=False, batch_size=64)

Found 20441 images belonging to 10 classes.
Found 1983 images belonging to 10 classes.


In [5]:
conv_mod = Model(input=mod.model.input, output=mod.model.layers[max_conv_layer].output)

In [8]:
conv_features = conv_mod.predict_generator(trn_batches, trn_batches.nb_sample)

In [9]:
conv_features.shape

(20441, 512, 14, 14)

In [10]:
save_array("conv_features.dat", conv_features)

In [8]:
conv_features = load_array("conv_features.dat")

In [12]:
conv_features.shape

(20441, 512, 14, 14)

In [13]:
conv_val_features = conv_mod.predict_generator(val_batches, val_batches.nb_sample)

In [9]:
# save_array("conv_val_features.dat", conv_val_features)
conv_val_features = load_array("conv_val_features.dat")

In [15]:
trn_labels = onehot(trn_batches.classes)
val_labels = onehot(val_batches.classes)

In [16]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes("data/")

Found 20441 images belonging to 10 classes.
Found 1983 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [17]:
trn_labels.shape

(20441, 10)

In [19]:
p = .8
inp = Input(shape=conv_mod.layers[-1].output_shape[1:])
mp = MaxPooling2D((2,2))(inp)
ft = Flatten()(mp)
dr1 = Dropout(p/2)(ft)
dn1 = Dense(128, activation='relu')(dr1)
bn1 = BatchNormalization()(dn1)
dr2 = Dropout(p/2)(bn1)
dn2 = Dense(128, activation='relu')(dr2)
bn2 = BatchNormalization()(dn2)
dr3 = Dropout(p)(bn2)
dn3 = Dense(10, activation='softmax')(dr3)

dense_mod = Model(input=inp, output=dn3)

In [20]:
opt = Adam(lr=.001)
dense_mod.compile(optimizer=opt, loss='categorical_crossentropy')

In [21]:
dense_mod.fit(conv_features, 
              trn_labels, 
              batch_size=64, 
              nb_epoch=10, 
              validation_data=(conv_val_features, val_labels))

Train on 20441 samples, validate on 1983 samples
Epoch 1/10
20441/20441 [==============================] - 6s - loss: 1.4803 - val_loss: 1.1191
Epoch 2/10
20441/20441 [==============================] - 6s - loss: 0.2737 - val_loss: 1.3544
Epoch 3/10
20441/20441 [==============================] - 6s - loss: 0.1315 - val_loss: 1.0569
Epoch 4/10
20441/20441 [==============================] - 6s - loss: 0.0948 - val_loss: 1.2024
Epoch 5/10
20441/20441 [==============================] - 6s - loss: 0.0624 - val_loss: 2.0449
Epoch 6/10
20441/20441 [==============================] - 6s - loss: 0.0541 - val_loss: 1.6194
Epoch 7/10
20441/20441 [==============================] - 6s - loss: 0.0466 - val_loss: 1.7684
Epoch 8/10
20441/20441 [==============================] - 6s - loss: 0.0407 - val_loss: 1.5232
Epoch 9/10
20441/20441 [==============================] - 6s - loss: 0.0421 - val_loss: 2.3329
Epoch 10/10
20441/20441 [==============================] - 6s - loss: 0.0335 - val_loss: 2.0566


## Data Augmentation

#### Below is an experiment to see if I get similar results to Jeremy

In [2]:
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [3]:
conv_model = Sequential(conv_layers)

In [223]:
conv_feat = conv_model.predict_generator(trn_batches, trn_batches.nb_sample)

In [6]:
val_conv_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [226]:
save_array("jeremy_conv_feat.dat", conv_feat)

In [7]:
save_array("jeremy_val_conv_feat.dat", val_conv_feat)

In [4]:
conv_feat = load_array("jeremy_conv_feat.dat")

In [8]:
val_conv_feat = load_array("jeremy_val_conv_feat.dat")

In [115]:
p = .8
inp = Input(shape=conv_model.layers[-1].output_shape[1:])
mp = MaxPooling2D((2,2))(inp)
ft = Flatten()(mp)
dr1 = Dropout(p)(ft)
dn1 = Dense(128, activation='relu')(dr1)
bn1 = BatchNormalization()(dn1)
dr2 = Dropout(p)(bn1)
dn2 = Dense(128, activation='relu')(dr2)
bn2 = BatchNormalization()(dn2)
dr3 = Dropout(p)(bn2)
dn3 = Dense(10, activation='softmax')(dr3)

dense_mod = Model(input=inp, output=dn3)

In [123]:
opt = Adam(lr=0.00001)
dense_mod.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [124]:
dense_mod.fit(conv_feat, trn_labels, batch_size=64, nb_epoch=10, validation_data=(val_conv_feat, val_labels))

Train on 20441 samples, validate on 1983 samples
Epoch 1/10
20441/20441 [==============================] - 4s - loss: 0.2155 - acc: 0.9342 - val_loss: 1.1332 - val_acc: 0.6722
Epoch 2/10
20441/20441 [==============================] - 4s - loss: 0.2100 - acc: 0.9341 - val_loss: 1.1204 - val_acc: 0.6737
Epoch 3/10
20441/20441 [==============================] - 4s - loss: 0.2111 - acc: 0.9355 - val_loss: 1.0957 - val_acc: 0.6858
Epoch 4/10
20441/20441 [==============================] - 4s - loss: 0.2058 - acc: 0.9368 - val_loss: 1.1042 - val_acc: 0.6808
Epoch 5/10
20441/20441 [==============================] - 4s - loss: 0.2088 - acc: 0.9349 - val_loss: 1.1040 - val_acc: 0.6813
Epoch 6/10
20441/20441 [==============================] - 4s - loss: 0.2039 - acc: 0.9358 - val_loss: 1.1077 - val_acc: 0.6823
Epoch 7/10
20441/20441 [==============================] - 4s - loss: 0.2024 - acc: 0.9374 - val_loss: 1.1002 - val_acc: 0.6838
Epoch 8/10
20441/20441 [==============================] - 4s -

In [25]:
opt = Adam(lr=0.0001)
dense_mod.compile(optimizer=opt, loss="categorical_crossentropy")

In [26]:
dense_mod.fit(conv_feat, trn_labels, batch_size=64, validation_data=(val_conv_feat, val_labels))

Train on 20441 samples, validate on 1983 samples
Epoch 1/10
20441/20441 [==============================] - 4s - loss: 0.1692 - val_loss: 1.3608
Epoch 2/10
20441/20441 [==============================] - 4s - loss: 0.1520 - val_loss: 1.3010
Epoch 3/10
20441/20441 [==============================] - 4s - loss: 0.1378 - val_loss: 1.3464
Epoch 4/10
20441/20441 [==============================] - 4s - loss: 0.1297 - val_loss: 1.3224
Epoch 5/10
20441/20441 [==============================] - 4s - loss: 0.1351 - val_loss: 1.3344
Epoch 6/10
20441/20441 [==============================] - 4s - loss: 0.1237 - val_loss: 1.3231
Epoch 7/10
12608/20441 [=================>............] - ETA: 1s - loss: 0.1143

KeyboardInterrupt: 